<a href="https://colab.research.google.com/github/idutsu/kirikuchikun-ai/blob/main/src/mecab-wiki-a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!apt-get install -y mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3 pandas
!pip install unidic
!python -m unidic download

In [ ]:
import os
import csv
import glob
import html
import json
import pandas as pd
import MeCab
from tqdm import tqdm

# MeCabを使ってWikipedia本文から名詞を抽出してCSVに書き込む関数
def extract_nouns_mecab(directory, output_csv, max_articles=None, max_nouns_per_row=10):
    """
    WikipediaのJSONファイルを処理し、名詞をCSVに書き出す関数。

    Parameters:
    - directory (str): JSONファイルが格納されているディレクトリ
    - output_csv (str): 出力CSVファイルのパス
    - max_articles (int, optional): 処理する最大記事数
    - max_nouns_per_row (int): 1行に書き込む名詞の最大数
    """
    tagger = MeCab.Tagger()

    # 既存のCSVから処理済みArticle_IDを取得
    processed_ids = set()
    if os.path.exists(output_csv):
        print("既存のCSVファイルを読み込み中...")
        try:
            df = pd.read_csv(output_csv, usecols=['Article_ID'], dtype={'Article_ID': str})
            processed_ids = set(df['Article_ID'].unique())
            print(f"処理済みID数: {len(processed_ids)}")
        except Exception as e:
            print(f"CSVファイル読み込みエラー: {e}")

    # JSONファイルの検索
    json_files = glob.glob(os.path.join(directory, '**', '*.json'), recursive=True)
    print(f"見つかったJSONファイルの数: {len(json_files)}")

    # CSVに結果を書き込み
    with open(output_csv, 'a', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        if os.stat(output_csv).st_size == 0:
            csv_writer.writerow(['Article_ID', 'Title', 'Nouns'])

        articles_processed = 0  # 処理した記事数のカウンタ

        for json_file in tqdm(json_files, desc="JSONファイルの処理"):
            try:
                with open(json_file, 'r', encoding='utf-8') as f:
                    for line in f:
                        if max_articles and articles_processed >= max_articles:
                            print("指定された記事数に達しました。処理を終了します。")
                            return

                        article = json.loads(line)
                        article_id = str(article.get('id', ''))
                        if article_id in processed_ids:  # 処理済みIDはスキップ
                            continue

                        title = article.get('title', '')
                        text = html.unescape(article.get('text', ''))

                        # MeCabで形態素解析し、名詞を抽出
                        nouns = []
                        for line in tagger.parse(text).splitlines():
                            if not line.strip() or line == 'EOS':  # 空行やEOSをスキップ
                                continue
                            parts = line.split('\t')
                            if len(parts) < 2:
                                continue
                            features = parts[1].split(',')
                            if features[0] == '名詞':  # 名詞の判定
                                nouns.append(parts[0])

                        # 名詞リストを複数行に分割して書き込み
                        for i in range(0, len(nouns), max_nouns_per_row):
                            chunk = nouns[i:i + max_nouns_per_row]
                            csv_writer.writerow([article_id, title, ', '.join(chunk)])

                        processed_ids.add(article_id)
                        articles_processed += 1

            except Exception as e:
                print(f"ファイル {json_file} の処理中にエラーが発生しました: {e}")

    print(f"処理が完了しました。結果は {output_csv} に保存されています。")

# 関数を実行
input_directory = "/content/drive/MyDrive/kirikuchikun-ai/wikipedia_json_output"  # JSONファイルが格納されているディレクトリ
output_csv = "/content/drive/MyDrive/kirikuchikun-ai/csv/wikipedia-a.csv"    # 出力CSVの保存先
extract_nouns_mecab(input_directory, output_csv)


In [11]:
#CSVファイルを読み込む
df = pd.read_csv(output_csv)

In [ ]:
# CSVファイルの記事数をカウント
try:
    # Article_IDの種類をカウント
    unique_ids = df['Article_ID'].nunique()
    print(f"CSVファイルに含まれるArticle_IDの種類: {unique_ids}")

except Exception as e:
    print(f"エラーが発生しました: {e}")


In [ ]:
# 任意の行を抜き出す
target_row = 1000000  # 例: 10行目を抜き出す

# 任意の行を表示
print(f"{target_row} 行目のデータ:")
print(df.iloc[target_row])

In [ ]:
# 任意の記事の名詞を全て抜き出す

target_id = 1  # 確認したいArticle_ID

# CSVファイルを読み込む
try:
    # 対象Article_IDの行をフィルタリング
    filtered_rows = df[df['Article_ID'] == target_id]

    # Nouns列の名詞を全て結合してリストにまとめる
    all_nouns = []
    for row in filtered_rows['Nouns'].dropna():  # Nouns列が空でない場合のみ処理
        nouns_list = row.split(',')  # カンマ区切りの名詞をリストに分割
        nouns_list = [noun.strip() for noun in nouns_list]  # 空白を取り除く
        all_nouns.extend(nouns_list)

    # 結果を表示
    if all_nouns:
        print(f"Article_ID {target_id} の名詞一覧:")
        print(all_nouns)
    else:
        print(f"Article_ID {target_id} の名詞は見つかりませんでした。")

except Exception as e:
    print(f"エラーが発生しました: {e}")
